In [19]:
%load_ext autoreload
%autoreload 2

import sys, os, shutil
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb

sys.path.append(os.path.join(os.getcwd(), '../symlie'))
from misc.utils import NumpyUtils, Results

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
log_dir = '../logs'
store_dir = os.path.join(log_dir, 'store')
wandb_dir = os.path.join(log_dir, 'wandb')

In [21]:
results_df_old = pd.read_pickle('../logs/store/results_df.pkl')
print(results_df_old.shape)
print(results_df_old.columns)
results_df_old.head()

(80, 39)
Index(['run_id', 'lr', 'net', 'bias', 'name', 'seed', 'tags', 'test', 'n_val',
       'train', 'y_low', 'device', 'n_test', 'y_high', 'log_dir', 'n_train',
       'predict', 'version', 'data_dir', 'eps_mult', 'n_splits', 'do_return',
       'noise_std', 'only_flip', 'batch_size', 'max_epochs', 'data_kwargs',
       'num_workers', 'space_length', 'generate_data', 'model_summary',
       'args_processed', 'do_return_model', 'transform_kwargs',
       'persistent_workers', 'run_name', 'test_loss', 'logger', 'grid_size'],
      dtype='object')


,run_id,lr,net,bias,name,seed,tags,test,n_val,train,...,generate_data,model_summary,args_processed,do_return_model,transform_kwargs,persistent_workers,run_name,test_loss,logger,grid_size
0,2m7eu5tn,0.0010,TrainP,False,None,5.0,[v5],True,100.0,True,...,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,worldly-snowflake-183,0.000473,NaN,NaN
1,ugylvzz6,0.0001,TrainP,False,None,5.0,[v5],True,100.0,True,...,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,classic-glade-184,0.009489,NaN,NaN
2,unpw27ml,0.1000,TrainP,False,None,5.0,[v5],True,100.0,True,...,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,sage-river-182,0.003589,NaN,NaN
3,w4yjcu25,0.0100,TrainP,False,None,5.0,[v5],True,100.0,True,...,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,azure-grass-181,0.003185,NaN,NaN
4,q86f7s5e,0.0001,TrainP,False,None,5.0,[v5],True,100.0,True,...,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,blooming-star-180,0.023643,NaN,NaN


In [17]:
api = wandb.Api()
filters = {'state': 'finished'}
runs = api.runs('eliasdubbeldam/symlie', filters=filters)
run_id_api = {run.id : {'tags':run.tags, 'name':run.name} for run in runs}

In [18]:
run_dirs =  np.array([[run_dir.split('-')[-1], run_dir] for run_dir in os.listdir(wandb_dir) if run_dir.startswith('run')])
if not len(run_dirs) == 0:
    assert len(run_dirs[:, 0]) == len(set(run_dirs[:, 0]))
    run_dirs = {run_id: run_dir for run_id, run_dir in run_dirs}
    run_dirs_df = pd.DataFrame(run_dirs.items(), columns = ['run_id', 'run_dir']).sort_values(by = 'run_dir', ascending = False)
else:
    run_dirs_df = pd.DataFrame(columns = ['run_id', 'run_dir'])


for i, values in run_dirs_df.iterrows():
    run_id = values['run_id']
    run_dir = values['run_dir']
    if run_id not in run_id_api:
        print(f'Removing {run_id}, {run_dir}')
        # shutil.rmtree(os.path.join(wandb_dir, values['run_dir']), )

run_dirs_df

Removing yf583iy8, run-20240129_152843-yf583iy8
Removing 2yffuyyh, run-20240125_153218-2yffuyyh
Removing mzjurctg, run-20240125_153211-mzjurctg
Removing kqsdmwsd, run-20240125_153207-kqsdmwsd
Removing uguvwf25, run-20240125_153206-uguvwf25
Removing h78ny35l, run-20240125_153205-h78ny35l
Removing mhzkk0i3, run-20240125_153203-mhzkk0i3
Removing i2klyk2q, run-20240125_153203-i2klyk2q
Removing h5g6wqls, run-20240125_153203-h5g6wqls
Removing 9xu71cw3, run-20240125_153203-9xu71cw3
Removing 6q06sgky, run-20240125_153203-6q06sgky
Removing 1pdj0ehk, run-20240125_153203-1pdj0ehk
Removing colf2qi0, run-20240125_153201-colf2qi0
Removing n103ix5y, run-20240125_153005-n103ix5y
Removing 90hd4qbd, run-20240125_153005-90hd4qbd
Removing v2f6nbkh, run-20240125_153002-v2f6nbkh
Removing n6o8bymo, run-20240125_153002-n6o8bymo
Removing hcurlkrr, run-20240125_153002-hcurlkrr
Removing 3ups9zx0, run-20240125_153002-3ups9zx0
Removing u6j7x5yh, run-20240125_153001-u6j7x5yh
Removing uthwe5o5, run-20240125_152959-u

,run_id,run_dir
68,uegf6m2h,run-20240129_164256-uegf6m2h
154,vz4jfzh6,run-20240129_163734-vz4jfzh6
140,wqmfolq3,run-20240129_163730-wqmfolq3
151,5k72ln34,run-20240129_163725-5k72ln34
54,mg3t7tf9,run-20240129_163226-mg3t7tf9
...,...,...
38,roz9fm0v,run-20240125_135752-roz9fm0v
173,wb53kqmt,run-20240125_135751-wb53kqmt
105,gouerrhp,run-20240125_135751-gouerrhp
192,e60auxdz,run-20240125_135751-e60auxdz


In [9]:
Ps = os.listdir(os.path.join(store_dir, 'P'))
for P in Ps:
    P_id = P.split('.')[0]
    if P_id not in run_id_api:
        print(f'Removing {P_id}')
        # os.remove(os.path.join(store_dir, 'P', P))

Removing yf583iy8


In [10]:
exclude_ids = []

In [23]:
tags = 'sine1d-predict'

api = wandb.Api()
filters = {'tags': tags} if tags else {}
filters['state'] = 'finished'
runs = api.runs('eliasdubbeldam/symlie', filters=filters)
print(len(runs))
for i, run in enumerate(runs):
    print(run.name, run.id, run.tags)
    # if run.id in exclude_ids: run.delete()

3
wise-terrain-249 03sba2gu ['sine1d-predict']
pious-shadow-249 4bxjjhy5 ['sine1d-predict']
breezy-shadow-249 miulsn4h ['sine1d-predict']


In [12]:
listdir_dirs = lambda path: [entry for entry in os.listdir(path) if os.path.isdir(os.path.join(path, entry))]
store_dir_dirs = listdir_dirs(store_dir)
store_dir_dirs

['P']

In [30]:
config_list = []

pbar = tqdm(runs)
for run in pbar:
    pbar.set_description(f'Retreiving wandb {str(id)}')
    
    id = run.id

    # Exceptions
    if run.state != 'finished':
        continue
    if id in results_df_old['run_id'].values:
        continue
    if id in exclude_ids:
        continue

    # print(id)
    # continue

    # Retreive config from wandb run, add all info to config
    config = run.config

    # Add run id and name to config
    config['run_id'] = id
    config['run_name'] = run.name

    # Retreive metrics from wandb run
    history = run.history()
    metrics = ['test_loss'] # 'train_loss', 'val_loss'
    for metric in metrics:

        config[metric] = np.array(history[metric].dropna()).item()

    config_list.append(config)

results_df = pd.DataFrame(config_list)
results_df

Retreiving wandb 4bxjjhy5: 100%|██████████| 3/3 [00:00<00:00,  3.59it/s]


,lr,net,bias,name,seed,tags,test,n_val,train,y_low,...,data_kwargs,num_workers,generate_data,model_summary,args_processed,do_return_model,transform_kwargs,persistent_workers,run_name,test_loss
0,0.001,Predict-NoneP,False,None,3,[sine1d-predict],True,1000,True,1,...,"{'y_low': 1, 'y_high': 3, 'grid_size': [1, 7],...",18,False,False,True,False,"{'eps_mult': [0, 0, 1, 0], 'only_flip': False}",True,wise-terrain-249,0.169928
1,0.001,Predict-NoneP,False,None,4,[sine1d-predict],True,1000,True,1,...,"{'y_low': 1, 'y_high': 3, 'grid_size': [1, 7],...",18,False,False,True,False,"{'eps_mult': [0, 0, 1, 0], 'only_flip': False}",True,pious-shadow-249,0.176168
2,0.001,Predict-NoneP,False,None,2,[sine1d-predict],True,1000,True,1,...,"{'y_low': 1, 'y_high': 3, 'grid_size': [1, 7],...",18,False,False,True,False,"{'eps_mult': [0, 0, 1, 0], 'only_flip': False}",True,breezy-shadow-249,0.154943


In [16]:
results_df_new = pd.concat([results_df_old, results_df]).reset_index(drop=True)
results_df_new.to_pickle(os.path.join(store_dir, 'results_df.pkl'))